## Further training an LLPR ensemble

Now, we will walk you through a process where you train an MLIP, then also the LLPR module associated with it for uncertainty quantification. Let's first generate the set of structures and obtain the reference energies:

To avoid re-training the model, we will just copy the trained model from the example in `1_training/`, along with the dataset it was trained on and the evaluation data.

```bash
cp ../1_training/model.ckpt .
cp ../1_training/dataset.xyz .
cp ../1_training/evaluation.xyz .
```

Assuming you have trained a model normally, you can now run the LLPR options from the `metatrain` CLI. What triggers the further training of the LLPR ensemble (after its creation) is the `num_epochs` keyword.

```bash
mtt train options-llpr.yaml -o model-llpr.pt
```

You can see that training is more expensive compared to the LLPR, but still quite cheap because we're only fine-tuning the last-layer ensemble weights.

After the model training finishes, you can evaluate the model on the held out dataset:
```bash
mtt eval model-llpr.pt eval.yaml -b 20
```

Let's now plot the spread of actual errors vs. the predicted errors from the LLPR formalism:

In [ ]:
# Load the structures
structures = ase.io.read("evaluation.xyz", ":")
evaluated_structures = ase.io.read("output.xyz", ":")

# Obtain the predictions and uncertainties from the evaluated structures
predicted_energies = [s.get_potential_energy() for s in evaluated_structures]
true_energies = [s.get_potential_energy() for s in structures]
errors = np.abs(np.array(predicted_energies) - np.array(true_energies))
uncertainties = [s.info["energy_uncertainty"] for s in evaluated_structures]
ensemble_uncertainties = [s.info["energy_ensemble"].std() for s in evaluated_structures]

quantile_lines = [0.00916, 0.10256, 0.4309805, 1.71796, 2.5348, 3.44388]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot using normal uncertainties
min_val_x, max_val_x = np.min(uncertainties), np.max(uncertainties)
min_val_y, max_val_y = np.min(errors), np.max(errors)
ax[0].plot([min_val_x, max_val_x], [min_val_x, max_val_x], "k--", lw=0.75)
for factor in quantile_lines:
    ax[0].plot([min_val_x, max_val_x], [factor * min_val_x, factor * max_val_x], "k:", lw=0.75)
ax[0].scatter(uncertainties, errors)
ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set_xlabel("Predicted energy uncertainty (eV)")
ax[0].set_ylabel("Absolute error in predicted energy (eV)")
ax[0].set_title("From LLPR")
ax[0].grid()

# Plot using ensemble uncertainties
min_val_x, max_val_x = np.min(ensemble_uncertainties), np.max(ensemble_uncertainties)
min_val_y, max_val_y = np.min(errors), np.max(errors)
ax[1].plot([min_val_x, max_val_x], [min_val_x, max_val_x], "k--", lw=0.75)
for factor in quantile_lines:
    ax[1].plot([min_val_x, max_val_x], [factor * min_val_x, factor * max_val_x], "k:", lw=0.75)
ax[1].scatter(ensemble_uncertainties, errors)
ax[1].set_xscale("log")
ax[1].set_yscale("log")
ax[1].set_xlabel("Predicted energy ensemble std (eV)")
ax[1].set_ylabel("Absolute error in predicted energy (eV)")
ax[1].set_title("From LLPR-derived shallow ensemble")
ax[1].grid()


You can hopefully see that the quality of the uncertainty predictions has improved.